In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#wells detection


In [ ]:
!pip install rasterio

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageChops, ImageEnhance
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks
from matplotlib.patches import Circle
from shapely.geometry import Point
from skimage.segmentation import slic
from skimage.measure import regionprops
import pandas as pd
import cv2

In [ ]:
# Function to load the image
def load_image(image_path):
    image = Image.open(image_path)
    return image

# Function to resize the image
def resize_image(image, target_size):
    image = image.resize(target_size, resample=Image.LANCZOS)
    return image

# Function to normalize the image
def normalize_image(image):
    image_array = np.array(image)
    normalized_image = image_array.astype(np.float32) / 255.0
    return normalized_image

# Function to enhance the contrast of the image
def enhance_contrast(image):
    enhanced_image = ImageEnhance.Contrast(image).enhance(1.5)
    return enhanced_image


# Function for image segmentation using SLIC algorithm
def image_segmentation(image, n_segments=1000, compactness=10, sigma=1):
    segments = slic(image, n_segments=n_segments, compactness=compactness, sigma=sigma)
    return segments

# Function to extract features for each segment
def extract_features_for_segments(segments, image):
    segment_props = regionprops(segments)
    features_list = []

    for segment in segment_props:
        segment_id = segment.label

        # Feature 1: Mean Color
        mean_color = np.mean(image[segments == segment_id], axis=0)

        # Feature 2: Standard Deviation of Color
        std_color = np.std(image[segments == segment_id], axis=0)

        # Feature 3: Area (Number of Pixels in the Segment)
        area = segment.area

        # Feature 4: Perimeter of the Segment
        perimeter = segment.perimeter

        # Create a dictionary of features for this segment
        segment_features = {
            'SegmentID': segment_id,
            'MeanColor': mean_color.tolist(),
            'StdColor': std_color.tolist(),
            'Area': area,
            'Perimeter': perimeter
        }

        # Append the features of this segment to the list
        features_list.append(segment_features)

    # Convert the list of dictionaries to a Pandas DataFrame for easy manipulation
    features_df = pd.DataFrame(features_list)

    return features_df

In [ ]:
# Function to z-score normalize data
def z_score_normalize(data):
    mean_value = data.mean()
    std_value = data.std()
    normalized_data = (data - mean_value) / std_value
    return normalized_data

In [ ]:
# Load the satellite image
file_path = "/content/drive/MyDrive/FarmWise AI/CV Assignment with Dataset (1)/CV Assignment with Dataset/satellite_image/kariyamangalam_clipped_raster.tif"
with rasterio.open(file_path) as src:
    data = src.read()
    metadata = src.meta

In [ ]:
# Preprocessing
target_size = (512, 512)
image = load_image(file_path)
image = resize_image(image, target_size)
image = enhance_contrast(image)
normalized_image = normalize_image(image)

# Image Segmentation
n_segments = 1000
compactness = 10
sigma = 1
segments = image_segmentation(normalized_image, n_segments=n_segments, compactness=compactness, sigma=sigma)

# Feature Extraction
features_df = extract_features_for_segments(segments, normalized_image)

# Normalize the pixel values for each band
for band_number in range(metadata["count"]):
    data[band_number] = z_score_normalize(data[band_number])

# Save the normalized data to a new raster file
normalized_file = "/content/drive/MyDrive/FarmWise AI/CV Assignment with Dataset (1)/CV Assignment with Dataset/satellite_image/normalized_raster.tif"
with rasterio.open(normalized_file, "w", **metadata) as dst:
    dst.write(data)

In [ ]:
# Loading normalized raster image
with rasterio.open(normalized_file) as src:
    normalized_data = src.read()
    metadata = src.meta
    # Choose a particular band
    band_to_segment = 1
    normalized_band_data = normalized_data[band_to_segment]
    # Convert the normalized band data to grayscale
    gray_band_data = (normalized_band_data * 255).astype(np.uint8)

# Enhance contrast
equalized_band_data = cv2.equalizeHist(gray_band_data)

# Sobel edge detection
sobel_x = cv2.Sobel(equalized_band_data, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(equalized_band_data, cv2.CV_64F, 0, 1, ksize=3)
gradient_magnitude = np.sqrt(sobel_x ** 2 + sobel_y ** 2)

# Define parameters
min_radius = 8
max_radius = 70
canny_sigma = 4.0

# Canny edge detection on the gradient magnitude image
edges = canny(gradient_magnitude, sigma=canny_sigma)

# Detecting circles - Hough Circle Transform
hough_radii = np.arange(min_radius, max_radius + 1)
hough_res = hough_circle(edges, hough_radii)

# Finding most prominent circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=40)

In [ ]:
# Load the satellite image using rasterio
with rasterio.open(file_path) as src:
    satellite_image = src.read()

# Create a geopandas GeoDataFrame to store the detected circles (wells)
gdf_wells = gpd.GeoDataFrame(columns=['geometry'])

# Convert the pixel coordinates of detected circles (wells) to geographical coordinates
for center_y, center_x, radius in zip(cy, cx, radii):
    # Convert pixel coordinates to geographical coordinates
    x_geo, y_geo = src.xy(center_y, center_x)

    # Create a Point geometry representing the circle center in the GeoDataFrame
    circle_center = Point(x_geo, y_geo)

    # Add the circle center to the GeoDataFrame for wells
    gdf_wells = gdf_wells.append({'geometry': circle_center}, ignore_index=True)

# Save the GeoDataFrame as a shapefile for wells
output_wells_shapefile = "/content/drive/MyDrive/FarmWise AI/CV Assignment with Dataset (1)/CV Assignment with Dataset/satellite_image/wells_shape_file.shp"
gdf_wells.to_file(output_wells_shapefile)

# Plot the detected wells as blue circles on the satellite image
plt.imshow(satellite_image.transpose(1, 2, 0))
plt.title("Satellite Image with Wells")
plt.axis('off')

for center_y, center_x, radius in zip(cy, cx, radii):
    circle = Circle((center_x, center_y), radius, color='blue', linewidth=2, fill=False)
    plt.gca().add_patch(circle)

plt.show()

# Farm Boundaries

In [ ]:
!pip install numpy rasterio matplotlib scikit-image shapely opencv-python geopandas


In [34]:
import os
import numpy as np
import rasterio
from skimage.segmentation import slic
from skimage.measure import regionprops
from shapely.geometry import Polygon
import cv2

# Function to preprocess the satellite image
def preprocess_image(image):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Enhance contrast
    enhanced_image = cv2.equalizeHist(gray_image)

    return enhanced_image

import geopandas as gpd

def save_farm_boundaries_to_shapefile(boundaries, file_path):
    # Create a GeoDataFrame from the polygons
    gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(boundaries))

    # Set the CRS for the GeoDataFrame to EPSG:4326 - WGS 84
    gdf.crs = 'EPSG:4326'

    # Save as a shapefile
    gdf.to_file(file_path)

# Load the satellite image using rasterio
file_path = "/content/drive/MyDrive/FarmWise AI/CV Assignment with Dataset (1)/CV Assignment with Dataset/satellite_image/kariyamangalam_clipped_raster.tif"
with rasterio.open(file_path) as src:
    satellite_image = src.read()

# Preprocess the satellite image
preprocessed_image = preprocess_image(satellite_image.transpose(1, 2, 0))

# Edge detection using Canny edge detector
edges = cv2.Canny(preprocessed_image, threshold1=100, threshold2=200)

# Post-process the binary edges to obtain farm boundaries using contour detection
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter out contours with too few points
min_points = 4
valid_contours = [contour for contour in contours if len(contour) >= min_points]

# Extract the farm boundaries as polygons
farm_boundaries = [Polygon(np.squeeze(contour)) for contour in valid_contours]

# Save the detected farm boundaries as a shapefile
output_farm_shapefile = "/content/drive/MyDrive/FarmWise AI/CV Assignment with Dataset (1)/CV Assignment with Dataset/farm_boundaries.shp"
save_farm_boundaries_to_shapefile(farm_boundaries, output_farm_shapefile)
